In [1]:
import pandas as pd
import geopandas as gpd
import requests
import os
from geojson import FeatureCollection
import shapely
import uuid
import random

##### IMPORT LOCAL DATA


In [3]:
bars = pd.read_excel("nashville-bars-data.xlsx")
MAPBOX_TOKEN = os.getenv("MAPBOX_TOKEN")

In [4]:
def mapboxGeocode(params: dict = {}):
    baseUrl = "https://api.mapbox.com/search/geocode/v6/forward"
    req = requests.get(baseUrl, params=params)
    return req.json()


def getGeocodeCoordinates(result: FeatureCollection):
    return ",".join(([str(x) for x in result['features'][0]["geometry"]["coordinates"]]))


def getAddressCoordinates(address):
    mapbox_query = {
        "q": address,
        "access_token": MAPBOX_TOKEN,
        "country": 'US'
    }
    geocode_results = mapboxGeocode(mapbox_query)
    string_coordinates = getGeocodeCoordinates(geocode_results)
    return string_coordinates

In [5]:
bars['coordinates'] = bars["address"].apply(lambda x: getAddressCoordinates(x))
bars['longitude'] = bars["coordinates"].apply(lambda x: float(x.split(",")[0]))
bars['latitude'] = bars["coordinates"].apply(lambda x: float(x.split(",")[1]))

In [6]:
bars.to_json("nashville-bars-data.json", orient="records")

In [2]:
bars = pd.read_json("nashville-bars-data.json")

In [ ]:
bars['geometry'] = bars.apply(lambda x:  shapely.Point(
    [x['longitude'], x['latitude']]), axis=1)

In [7]:
bars_gdf = gpd.GeoDataFrame(bars, geometry="geometry")

In [8]:
zipcodes_gdf = gpd.read_file("nashville_zipcode_refined.json")

In [34]:
def getProFromIntersect(k: gpd.GeoSeries, t: gpd.GeoDataFrame, p: str):
    loc: list[shapely.Polygon | shapely.MultiPolygon] = list(t.geometry)
    count = 0
    found = False
    while not found:
        if (count == len(loc)):
            return None

        item_loc = loc[count]
        found = item_loc.intersects(k.geometry)
        if (found):
            break
        else:
            count += 1

    return t[p][count]

In [ ]:
bars_gdf['zipcode'] = bars_gdf.apply(
    lambda x: getProFromIntersect(x, zipcodes_gdf, "zipcode"), axis=1)

In [29]:
bars_gdf.to_file("nashville-bars-data.geojson", driver="GeoJSON")

##### CUSTOM NEIGHBOURHOOD


In [32]:
bars_gdf = gpd.read_file("nashville-bars-data.geojson")
hood_gpf = gpd.read_file("nashville-neighbourhoods-custom.geojson")

In [ ]:
bars_gdf['hood'] = bars_gdf.apply(
    lambda x: getProFromIntersect(x, hood_gpf, 'name'), axis=1)

In [39]:
bars_gdf.to_file("nashville-bars-data.geojson", driver="GeoJSON")

##### ADD ID


In [2]:
bars_gdf = gpd.read_file("nashville-bars-data.geojson")
hood_gpf = gpd.read_file("nashville-neighbourhoods-custom.geojson")

In [10]:
bars_gdf['id'] = bars_gdf.apply(lambda x: str(uuid.uuid4()), axis=1)
hood_gpf['id'] = hood_gpf.apply(lambda x: str(uuid.uuid4()), axis=1)

In [11]:
bars_gdf.to_file("nashville-bars-data.geojson", driver="GeoJSON")
hood_gpf.to_file("nashville-neighbourhoods-custom.geojson", driver="GeoJSON")

##### For wordpress database


In [2]:
bars_gdf = gpd.read_file("nashville-bars-data.geojson")
hood_gpf = gpd.read_file("nashville-neighbourhoods-custom.geojson")

In [6]:
bars_gdf.to_csv("nashville-bars-data-sql.csv", index=False)
hood_gpf.to_csv("nashville-hoods-data-sql.csv", index=False)

In [5]:
pd.read_csv("nashville-bars-data-sql.csv").to_json(
    "nashville-bars-data-sql.json", index=False, orient="records")
pd.read_csv("nashville-hoods-data-sql.csv").to_json("nashville-hoods-data-sql.json",
                                                    index=False, orient="records")

#### add categories


In [5]:
bars_gdf = pd.read_csv("nashville-bars-data-sql.csv")

In [6]:
prime_cat = ["Bar", "Restaurant", "Accomodation", "Live Music Venue"]
bars_sub_cat = ["Speekeasy", "Celebrity", "Dive Bar", "Classic"]
acco_sub_cat = ['AirBnB', "VRBO"]

In [7]:
bars_gdf['primary'] = random.choices(
    prime_cat, weights=[3, 1, 2, 1],  k=10)

In [8]:
def getSubCat(cat: str):
    if "Bar" == cat:
        return random.choice(bars_sub_cat)
    if "Accomodation" == cat:
        return random.choice(acco_sub_cat)
    return ""

In [9]:
bars_gdf['secondary'] = bars_gdf.apply(lambda x: getSubCat(x.primary), axis=1)

In [38]:
bars_gdf.to_file("nashville-bars-data.geojson", driver="GeoJSON")

In [2]:
bars_gdf = gpd.read_file("nashville-bars-data.geojson")

In [12]:
bars_gdf.to_csv('nashville-bars-data-sql.csv')

In [ ]:
icons = [
    {
        label: 'hotel-spot',
        url: 'https://raw.githubusercontent.com/Auh3b/nashville-map-data/refs/heads/main/hotel-sdf.png',
    },
    {
        label: 'bar-spot',
        url: 'https://raw.githubusercontent.com/Auh3b/nashville-map-data/refs/heads/main/beer-1-sdf.png',
    },
    {
        label: 'restaurant-spot',
        url: 'https://raw.githubusercontent.com/Auh3b/nashville-map-data/refs/heads/main/restaurant-sdf.png',
    },
    {
        label: 'live-event-spot',
        url: 'https://raw.githubusercontent.com/Auh3b/nashville-map-data/refs/heads/main/live-events-sdf.png',
    },
]